In [30]:
%reload_ext autoreload
%autoreload 2
import chess.pgn
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

from chess_dataset import ChessDataset, ChessPairDataset
import helper

import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision

In [31]:
train_chess_pair_dataset = ChessPairDataset("dataset/npy/")
test_chess_pair_dataset = ChessPairDataset("dataset/npy/", train=False)
train_chess_dataset = ChessDataset("dataset/npy/")
test_chess_dataset = ChessDataset("dataset/npy/", train=False)

In [32]:
len(train_chess_pair_dataset), len(test_chess_pair_dataset)

(80000, 20000)

In [33]:
len(train_chess_dataset), len(test_chess_dataset)

(2758, 690)

In [34]:
chess_trainloader = DataLoader(train_chess_dataset, batch_size=64, shuffle=True)
chess_testloader = DataLoader(test_chess_dataset, batch_size=64, shuffle=False)
chess_pair_trainloader = DataLoader(train_chess_pair_dataset, batch_size=64, shuffle=True)
chess_pair_testloader = DataLoader(test_chess_pair_dataset, batch_size=64, shuffle=True)

In [35]:
def freeze(layer):
  layer.weight.requires_grad = False
  layer.bias.requires_grad = False

def unfreeze(layer):
  layer.weight.requires_grad = True
  layer.bias.requires_grad = True
  
# Dense encoding layers
en1 = nn.Linear(773, 600)
en2 = nn.Linear(600, 400)
en3 = nn.Linear(400, 200)
en4 = nn.Linear(200, 100)

# Dense decoding layers
de1 = nn.Linear(600, 773)
de2 = nn.Linear(400, 600)
de3 = nn.Linear(200, 400)
de4 = nn.Linear(100, 200)

# Autoencoder train
# 773 - 600 - 773
ae1 = nn.Sequential(en1, nn.ReLU(), 
                    de1, nn.Sigmoid())
helper.train_autoencoder(ae1, chess_trainloader, chess_testloader, 10, 2)

# * means freeze
# 773 *- 600 - 400 - 600 *- 773
freeze(en1)
freeze(de1)
ae2 = nn.Sequential(en1, nn.ReLU(), en2, nn.ReLU(), 
                    de2, nn.ReLU(), de1, nn.Sigmoid())
helper.train_autoencoder(ae2, chess_trainloader, chess_testloader, 10, 2)

# 773 *- 600 *- 400 - 200 - 400 *- 600 *- 773
freeze(en2)
freeze(de2)
ae3 = nn.Sequential(en1, nn.ReLU(), en2, nn.ReLU(), en3, nn.ReLU(), 
                    de3, nn.ReLU(), de2, nn.ReLU(), de1, nn.Sigmoid())
helper.train_autoencoder(ae3, chess_trainloader, chess_testloader, 10, 2)

# 773 *- 600 *- 400 *- 200 - 100 - 200 *- 400 *- 600 *- 773
freeze(en3)
freeze(de3)
ae4 = nn.Sequential(en1, nn.ReLU(), en2, nn.ReLU(), en3, nn.ReLU(), en4, nn.ReLU(), 
                    de4, nn.ReLU(), de3, nn.ReLU(), de2, nn.ReLU(), de1, nn.Sigmoid())
helper.train_autoencoder(ae4, chess_trainloader, chess_testloader, 10, 2)

unfreeze(en1)
unfreeze(en1)
unfreeze(en3)
pos2vec = nn.Sequential(en1, nn.ReLU(), en2, nn.ReLU(), en3, nn.ReLU(), en4, nn.ReLU())

Epoch: 0 | Loss: 0.1979263871908188: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:00<00:00, 60.15it/s]
Validation ->  Loss: 0.20344744622707367: 100%|████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 72.42it/s]
Epoch: 1 | Loss: 0.060266993939876556: 100%|███████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:00<00:00, 95.24it/s]
Validation ->  Loss: 0.10949824005365372: 100%|███████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 124.79it/s]
Epoch: 2 | Loss: 0.06240382045507431: 100%|████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:00<00:00, 94.76it/s]
Validation ->  Loss: 0.06606277078390121: 100%|███████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 130.05it/s]
Epoch: 3 | Loss:

In [38]:
class DeepChess(nn.Module):
  def __init__(self, pos2vec):
    super(DeepChess, self).__init__()
    self.pos2vec = pos2vec
    self.base = nn.Sequential(nn.Linear(200, 400), nn.ReLU(), nn.BatchNorm1d(400), nn.Dropout(0.3),
                              nn.Linear(400, 200), nn.ReLU(), nn.BatchNorm1d(200), nn.Dropout(0.3),
                              nn.Linear(200, 100), nn.ReLU(), nn.BatchNorm1d(100), nn.Dropout(0.3),
                              nn.Linear(100, 2), nn.Softmax(dim=1))
  
  def forward(self, input1, input2):
    first_board = self.pos2vec(input1)
    second_board = self.pos2vec(input2)
    concat = torch.cat((first_board, second_board), 1)
    return self.base(concat)

In [39]:
dp = DeepChess(pos2vec)
dp

DeepChess(
  (pos2vec): Sequential(
    (0): Linear(in_features=773, out_features=600, bias=True)
    (1): ReLU()
    (2): Linear(in_features=600, out_features=400, bias=True)
    (3): ReLU()
    (4): Linear(in_features=400, out_features=200, bias=True)
    (5): ReLU()
    (6): Linear(in_features=200, out_features=100, bias=True)
    (7): ReLU()
  )
  (base): Sequential(
    (0): Linear(in_features=200, out_features=400, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=400, out_features=200, bias=True)
    (5): ReLU()
    (6): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=200, out_features=100, bias=True)
    (9): ReLU()
    (10): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Dropout(p=0.3, inplace=False)
   

In [40]:
helper.train_supervise(model=dp, train_data=chess_pair_trainloader, val_data=chess_pair_testloader, epochs=10, patient=2)

Epoch: 0 | Loss: 0.0006117189768701792 | Accuracy: 0.97165: 100%|██████████████████████████████████████████████████████████████| 1250/1250 [00:25<00:00, 48.40it/s]
Validation -> Loss: 1.053195595741272 | Accuracy: 0.68795: 100%|█████████████████████████████████████████████████████████████████| 313/313 [00:04<00:00, 65.19it/s]
Epoch: 1 | Loss: 0.00023676466662436724 | Accuracy: 1.0: 100%|█████████████████████████████████████████████████████████████████| 1250/1250 [00:25<00:00, 49.53it/s]
Validation -> Loss: 0.5434906482696533 | Accuracy: 0.6891: 100%|█████████████████████████████████████████████████████████████████| 313/313 [00:04<00:00, 65.47it/s]
Epoch: 2 | Loss: 0.00011599027493502945 | Accuracy: 0.999875: 100%|████████████████████████████████████████████████████████████| 1250/1250 [00:25<00:00, 49.29it/s]
Validation -> Loss: 1.4661428928375244 | Accuracy: 0.6574: 100%|█████████████████████████████████████████████████████████████████| 313/313 [00:04<00:00, 65.45it/s]
Epoch: 3 | Loss:

[tensor(1.0532, grad_fn=<BinaryCrossEntropyBackward0>),
 tensor(0.5435, grad_fn=<BinaryCrossEntropyBackward0>),
 tensor(1.4661, grad_fn=<BinaryCrossEntropyBackward0>),
 tensor(1.9275, grad_fn=<BinaryCrossEntropyBackward0>),
 tensor(1.1385, grad_fn=<BinaryCrossEntropyBackward0>),
 tensor(1.3920, grad_fn=<BinaryCrossEntropyBackward0>),
 tensor(1.5060, grad_fn=<BinaryCrossEntropyBackward0>),
 tensor(0.9788, grad_fn=<BinaryCrossEntropyBackward0>),
 tensor(0.5954, grad_fn=<BinaryCrossEntropyBackward0>),
 tensor(2.6879, grad_fn=<BinaryCrossEntropyBackward0>)]

In [43]:
i = next(iter(chess_pair_testloader))
z = i[2]
zz = dp(i[0], i[1])
print(torch.argmax(z, 1))
#print(zz)
print(torch.argmax(zz, 1))
# for a in zip(z, zz):
#   print(a)      
print(sum(1 for x in (torch.argmax(z, 1) == torch.argmax(zz, 1)) if x), "/", i[0].shape[0])

tensor([1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1,
        0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1])
tensor([0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
        0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1,
        0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1])
37 / 64


In [13]:
torch.save(dp.state_dict(), "model/save/deepchess.pth")
torch.save(pos2vec.state_dict(), "model/save/pos2vec.pth")